In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse 
from sklearn.preprocessing import StandardScaler

In [24]:
#set seed
np.random.seed(912)

# Sử dụng Softmax làm hàm đầu vào cho bài toán hồi quy

## Hàm softmax

In [25]:
def softmax_stable(Z):
    """
    Sử dụng softmax_stable để tránh tràn số  
    """
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A

def convert_labels(y, C):
    """
    Biến vector y thành ma trận one-hot
    """
    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y 

def loss_function(X, Y, W):
    """
    Tính hàm mất mát cross-entropy loss
    """
    A = softmax_stable(W.T.dot(X))
    return -np.sum(Y*np.log(A))

def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 50000):
    """
    Softmax Regression sử dụng Stochastic Gradient Descent (SGD)
    """
    W = [W_init]    
    C = W_init.shape[1]
    Y = convert_labels(y, C)
    it = 0
    N = X.shape[1]
    d = X.shape[0]
    
    count = 0
    check_w_after = 20
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax_stable(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta*xi.dot((yi - ai).T)
            count += 1
            # Điều kiện dừng
            if count%check_w_after == 0:                
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W

def pred(W, X, index):
    """
    Đầu vào cho bài toán hồi quy
    """
    A = softmax_stable(W.T.dot(X))
    return A[index]

## Lưu dữ liệu vào data

In [26]:
df = pd.read_csv('data/obesity_data_cleaned.csv')

In [27]:
X = df.drop('NObeyesdad', axis=1).values
y = df['NObeyesdad'].astype('category').cat.codes.values

In [28]:
continuous_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
continuous_idx = [df.columns.get_loc(col) for col in continuous_cols]
continuous_idx

[1, 2, 3, 6, 7, 9, 10, 11]

In [29]:
target_mapping = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Obesity_Type_I': 2,
    'Obesity_Type_II': 3,
    'Obesity_Type_III': 4,
    'Overweight_Level_I': 5,
    'Overweight_Level_II': 6
    }

In [30]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X[:, continuous_idx] = scaler.fit_transform(X[:, continuous_idx])

In [31]:
# Khởi tạo tham số và huấn luyện mô hình
d = X.shape[1] # Số đặc trưng
C = len(np.unique(y)) # Số lớp
W_init = np.random.randn(d, C) # Khởi tạo trọng số ngẫu nhiên

W = softmax_regression(X.T, y, W_init, eta=0.05)
W_final = W[-1]

y_lin = pred(W_final, X.T, target_mapping['Normal_Weight'])

y_lin

array([1.61693104e-02, 8.61908806e-01, 1.36696217e-01, ...,
       1.61720476e-05, 8.40533282e-03, 1.74327013e-10], shape=(20758,))

In [34]:
df_lin = pd.DataFrame(X, columns=df.drop('NObeyesdad', axis=1).columns)
df_lin['y'] = y_lin

df_lin.to_csv('data/obesity_regression_input.csv', index=False)